In [133]:
!pwd

/home/analytics/Memelysis/analytics


In [134]:
import findspark
findspark.init()

In [135]:
import subprocess

from pyspark import SparkContext, SparkConf

from pyspark.sql.types import StructType, IntegerType, StringType
from pyspark.sql import SparkSession, SQLContext, functions

In [136]:
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [137]:
cmd = 'hdfs dfs -ls /jsons/'

files = subprocess.check_output(cmd, shell=True)
files = files.strip()
files = files.decode().split('\n')

out = []

for path in files:
    f = path.split(" ")[-1]
    if 'memes_' in f:
        out.append(f)
        print(f)

/jsons/memes_1588956783893.json
/jsons/memes_1588975800125.json
/jsons/memes_1588979400070.json
/jsons/memes_1588983000044.json
/jsons/memes_1588986600066.json
/jsons/memes_1588990200082.json
/jsons/memes_1588993800056.json
/jsons/memes_1588997400057.json
/jsons/memes_1589001000045.json
/jsons/memes_1589004600051.json
/jsons/memes_1589008200048.json
/jsons/memes_1589011800040.json
/jsons/memes_1589015400180.json
/jsons/memes_1589019000033.json
/jsons/memes_1589022600061.json


In [138]:
sqlContext = SQLContext(sc)

In [139]:
sample_json = out[-1]

In [140]:
df = spark.read.json(sample_json)

In [141]:
df.printSchema()

root
 |-- additional_data: string (nullable = true)
 |-- extension: string (nullable = true)
 |-- id: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- url: string (nullable = true)



# TWITTER

In [142]:
twitter_df = df.filter(df.source == 'twitter')

twitter_schema = StructType().add(
    'created_at', StringType(), True).add(
    'text', StringType(), True).add(
    'favorite_count', IntegerType(), True).add(
    'retweet_count', IntegerType(), True).add(
    'hashtags', StringType(), True)

twitter_data =  twitter_df.select(
    functions.col('id'),
    functions.from_json(functions.col('additional_data'), schema=twitter_schema).alias("data")
).select('id', 'data.*')

twitter_data = twitter_data.withColumnRenamed('text', 'tweet_text')

twitter_df = twitter_df.alias('twitter_df')
twitter_data = twitter_data.alias('twitter_data')

inner_join = twitter_df.join(twitter_data,
                             twitter_df.id == twitter_data.id).select(
    'twitter_df.id',
    'twitter_df.image_path',
    'twitter_df.extension',
    'twitter_df.source',
    'twitter_df.text',
    'twitter_df.url',
    'twitter_data.created_at',
    'twitter_data.tweet_text',
    'twitter_data.favorite_count',
    'twitter_data.retweet_count',
    'twitter_data.hashtags')
inner_join.printSchema()
inner_join.show()

root
 |-- id: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- extension: string (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- hashtags: string (nullable = true)

+-------------------+--------------------+---------+-------+--------------------+--------------------+-------------------+--------------------+--------------+-------------+--------------------+
|                 id|          image_path|extension| source|                text|                 url|         created_at|          tweet_text|favorite_count|retweet_count|            hashtags|
+-------------------+--------------------+---------+-------+--------------------+--------------------+-------------------+--------------------+--------------+-----

# MEMEDROID

In [145]:
x = memedroid_df.select('additional_data').limit(1).collect()

In [148]:
memedroid_df = df.filter(df.source == 'memedroid')

memedroid_schema = StructType().add(
    'title', StringType(), True).add(
    'tags', StringType(), True).add(
    'date', IntegerType(), True).add(
    'popularity', IntegerType(), True)

memedroid_data =  memedroid_df.select(
    functions.col('id'),
    functions.from_json(functions.col('additional_data'), schema=memedroid_schema).alias("data")
).select('id', 'data.*')


memedroid_df = memedroid_df.alias('memedroid_df')
memedroid_data = memedroid_data.alias('memedroid_data')

inner_join = memedroid_df.join(memedroid_data,
                             memedroid_df.id == memedroid_data.id).select(
    'memedroid_df.id',
    'memedroid_df.image_path',
    'memedroid_df.extension',
    'memedroid_df.source',
    'memedroid_df.text',
    'memedroid_df.url',
    'memedroid_data.title',
    'memedroid_data.tags',
    'memedroid_data.date',
    'memedroid_data.popularity')

inner_join.printSchema()
inner_join.show()

root
 |-- id: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- extension: string (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- popularity: integer (nullable = true)

+--------------------+--------------------+---------+---------+--------------------+--------------------+-----+----+----+----------+
|                  id|          image_path|extension|   source|                text|                 url|title|tags|date|popularity|
+--------------------+--------------------+---------+---------+--------------------+--------------------+-----+----+----+----------+
|UPLOADED715eb4ba4...|images/memedroid_...|     jpeg|memedroid|REMEMBER WHEN YOU...|https://images3.m...| null|null|null|      null|
|UPLOADED9565eb509...|images/memedroid_...|     jpeg|memedroid|My microwave The ...

# IMGUR

In [151]:
imgur_df = df.filter(df.source == 'imgur')
x = imgur_df.select('additional_data').limit(1).collect()
print(x)
imgur_df.show()

[Row(additional_data='{"id":"2RXU3z2","title":"The risky move","description":null,"datetime":1589011180,"cover":"nPtkoem","cover_width":460,"cover_height":570,"account_url":"KrugerSchmidts","account_id":14100021,"privacy":"hidden","layout":"blog","views":134,"link":"https://imgur.com/a/2RXU3z2","ups":4,"downs":5,"points":-1,"score":0,"is_album":true,"vote":null,"favorite":false,"nsfw":false,"section":"","comment_count":4,"favorite_count":0,"topic":"No Topic","topic_id":29,"images_count":1,"in_gallery":true,"is_ad":false,"tags":[{"name":"memes","display_name":"memes","followers":1898119,"total_items":520012,"following":false,"is_whitelisted":false,"background_hash":"1z4dgZT","thumbnail_hash":null,"accent":"ba519f","background_is_animated":false,"thumbnail_is_animated":false,"is_promoted":false,"description":"the freshest memes","logo_hash":null,"logo_destination_url":null,"description_annotations":[]},{"name":"chess","display_name":"chess","followers":383,"total_items":995,"following":f

In [157]:
imgur_df = df.filter(df.source == 'imgur')

imgur_schema = StructType().add(
    'title', StringType(), True).add(
    'datetime', IntegerType(), True).add(
    'views', IntegerType(), True).add(
    'ups', IntegerType(), True).add(
    'downs', IntegerType(), True).add(
    'points', IntegerType(), True).add(
    'nsfw', StringType(), True)
imgur_data =  imgur_df.select(
    functions.col('id'),
    functions.from_json(functions.col('additional_data'), schema=imgur_schema).alias("data")
).select('id', 'data.*')

imgur_df = imgur_df.alias('imgur_df')
imgur_data = imgur_data.alias('imgur_data')

inner_join = imgur_df.join(imgur_data,
                             imgur_df.id == imgur_data.id).select(
    'imgur_df.id',
    'imgur_df.image_path',
    'imgur_df.extension',
    'imgur_df.source',
    'imgur_df.text',
    'imgur_df.url',
    'imgur_data.title',
    'imgur_data.datetime',
    'imgur_data.datetime',
    'imgur_data.datetime',
    'imgur_data.datetime',
    'imgur_data.datetime',
    )

inner_join.printSchema()
inner_join.show()

root
 |-- id: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- extension: string (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- datetime: integer (nullable = true)

+-------+--------------------+---------+------+--------------------+--------------------+--------------------+----------+
|     id|          image_path|extension|source|                text|                 url|               title|  datetime|
+-------+--------------------+---------+------+--------------------+--------------------+--------------------+----------+
|nPtkoem|images/imgur_2020...|      jpg| imgur|Students Dont be ...|https://i.imgur.c...|      The risky move|1589011180|
|0xd5nnG|images/imgur_2020...|      jpg| imgur|skettie @ben_gett...|https://i.imgur.c...|Some pet memes to...|1589009323|
|yxbpgtF|images/imgur_2020...|      jpg| imgur|I GUARD MY MASTER...|https://i.im